# Dynamische Audio-Ausrichtung für Chunks (Batch-Verarbeitung)

Dieses Notebook richtet ganze Ordner von sauberen Audio-Chunks (`clean`) dynamisch an entsprechende Tape-Output-Chunks (`tape`) aus. Es ist für die Batch-Verarbeitung optimiert.

**Ziel:** Den lokalen Zeitversatz (Lag), der durch Tape-Aufnahmen entsteht, für kurze Audio-Chunks zu korrigieren.

**Besonderheiten:**
- **Hohe Sample Rate (96kHz):** Das Skript arbeitet nativ mit 96kHz-Audiodaten.
- **Optimierte Lag-Berechnung:** Für die rechenintensive Kreuzkorrelation wird das Audio temporär auf eine niedrigere Sample Rate heruntergerechnet (`SR_CALC`), um die Performance zu steigern.
- **Skalierung:** Die berechneten Lag-Werte werden präzise auf die ursprüngliche 96kHz-Zeitbasis zurückskaliert, bevor das finale Time-Warping auf die hochauflösenden Daten angewendet wird.
- **Batch-Verarbeitung:** Das Skript durchsucht automatisch Input-Verzeichnisse, findet entsprechende Paare von Clean- und Tape-Chunks und verarbeitet sie nacheinander.
- **Validierung:** Für jeden Chunk wird der verbleibende Versatz nach der Korrektur berechnet und ausgegeben, um den Erfolg der Ausrichtung zu überprüfen.


## 1. Import und Konfiguration

In [ ]:
import numpy as np
import scipy.signal
from scipy.interpolate import interp1d
import librosa
import soundfile as sf
from typing import Tuple, List
from pathlib import Path
import os


### Globale Parameter
- **SR_ORIGINAL:** Die native Sample Rate der Quelldateien (z.B. 96000 Hz).
- **SR_CALC:** Eine niedrigere Sample Rate (z.B. 16000 Hz) für die performante Berechnung des Lags.
- **WINDOW_SEC / HOP_SEC:** Parameter für die fensterbasierte Analyse.
- **MIN_CHUNK_DURATION_SEC:** Schwellenwert, unter dem Chunks als zu kurz für eine Analyse angesehen und übersprungen werden.

In [ ]:
# --- Globale Konfiguration ---
SR_ORIGINAL = 96000  # Native Sample Rate der Audio-Dateien
SR_CALC = 16000      # Niedrigere Sample Rate für die schnelle Lag-Berechnung

WINDOW_SEC = 0.5     # Fenstergröße für die Analyse in Sekunden
HOP_SEC = 0.25       # Schrittweite (Overlap) für die Analyse in Sekunden

# Mindestlänge eines Chunks in Sekunden, um verarbeitet zu werden. Muss > WINDOW_SEC sein.
MIN_CHUNK_DURATION_SEC = 0.6


### Pfad-Konfiguration
Das Skript geht von einer parallelen Ordnerstruktur aus:
- `.../data/input/SET_NAME/CHUNK_NAME.wav` (Originale)
- `.../data/output/SET_NAME_tape/CHUNK_NAME.wav` (Tape-Versionen)
- `.../data/output/SET_NAME_aligned/CHUNK_NAME.wav` (Ergebnisdateien)

In [ ]:
# --- Pfad-Konfiguration ---
BASE_INPUT_DIR = Path("/Users/mischakurth/Documents/GitHub/mischakurth/xmas-hackathon-2025-bandsalat/data/audio/datasets/dataset-alfred/tape-input")
BASE_TAPE_DIR = Path("/Users/mischakurth/Documents/GitHub/mischakurth/xmas-hackathon-2025-bandsalat/data/audio/datasets/dataset-alfred/tape-output-recordings")
BASE_OUTPUT_DIR = Path("/Users/mischakurth/Documents/GitHub/mischakurth/xmas-hackathon-2025-bandsalat/data/audio/datasets/dataset-alfred/tape-input-warped")


## 2. Hilfsfunktionen

### `compute_local_lags_resampled`
Diese Funktion ist das Herzstück der Lag-Analyse. Sie nimmt die hochauflösenden Signale, resampelt sie auf `SR_CALC`, führt die fensterbasierte Kreuzkorrelation durch und skaliert die Ergebnisse (Zeitpunkte und Lag-Werte) wieder auf die `SR_ORIGINAL` hoch. Das spart erheblich Rechenzeit.

In [ ]:
def compute_local_lags_resampled(
    x_clean_orig: np.ndarray,
    x_tape_orig: np.ndarray,
    sr_orig: int,
    sr_calc: int,
    win_size_sec: float,
    hop_size_sec: float
) -> Tuple[np.ndarray, np.ndarray]:
    """
    Berechnet den lokalen Versatz bei einer niedrigeren Sample Rate, um die Performance zu verbessern.
    """
    # 1. Resample für die Berechnung
    resample_ratio = sr_calc / sr_orig
    x_clean_calc = librosa.resample(y=x_clean_orig, orig_sr=sr_orig, target_sr=sr_calc)
    x_tape_calc = librosa.resample(y=x_tape_orig, orig_sr=sr_orig, target_sr=sr_calc)

    # 2. Fenster-Parameter für die Berechnungs-Sample-Rate bestimmen
    win_size_calc = int(win_size_sec * sr_calc)
    hop_size_calc = int(hop_size_sec * sr_calc)

    lags_calc = []
    time_points_calc = []
    max_lag_search_calc = int(2000 * resample_ratio) # Skaliere auch den Suchbereich

    num_windows = int((len(x_tape_calc) - win_size_calc) / hop_size_calc)
    if num_windows == 0:
        return np.array([]), np.array([])

    for i in range(num_windows):
        start = i * hop_size_calc
        end = start + win_size_calc
        seg_clean = x_clean_calc[start:end]
        seg_tape = x_tape_calc[start:end]

        corr = scipy.signal.correlate(seg_tape, seg_clean, mode='same', method='fft')

        mid_point = len(corr) // 2
        search_start = max(0, mid_point - max_lag_search_calc)
        search_end = min(len(corr), mid_point + max_lag_search_calc)

        search_area = corr[search_start:search_end]
        if len(search_area) == 0: continue

        peak_idx_relative = np.argmax(search_area)
        current_lag = (search_start + peak_idx_relative) - mid_point

        lags_calc.append(current_lag)
        time_points_calc.append(start + win_size_calc // 2)

    # 3. Zeitpunkte und Lags zurück auf die originale Sample Rate skalieren
    scale_factor = sr_orig / sr_calc
    time_points_orig = np.array(time_points_calc) * scale_factor
    lags_orig = np.array(lags_calc) * scale_factor

    return time_points_orig, lags_orig


### `apply_warping_full_res`
Diese Funktion nimmt die hochskalierten Lag-Werte und wendet sie auf das *originale*, hochauflösende Signal an. Sie glättet zunächst die Lag-Kurve, interpoliert sie auf die volle Sample-Anzahl und führt dann das Time-Warping mittels kubischer Interpolation durch.

In [ ]:
def apply_warping_full_res(
    x_clean: np.ndarray,
    lags: np.ndarray,
    lag_times: np.ndarray
) -> np.ndarray:
    """
    Wendet die skalierten Lags auf das hochauflösende Originalsignal an.
    """
    total_length = len(x_clean)

    # Glätte die Lags, um Ausreißer zu entfernen.
    # Der Median-Filter ist robust gegen einzelne falsche Messungen.
    # Die Kernel-Größe muss ungerade sein.
    kernel_size = min(15, len(lags) - (1 if len(lags) % 2 == 0 else 0))
    if kernel_size < 1: kernel_size = 1
    lags_smooth = scipy.signal.medfilt(lags, kernel_size=kernel_size)

    # Interpoliere die geglätteten Lags auf die volle Länge des Signals.
    lag_interpolator = interp1d(lag_times, lags_smooth, kind='linear', fill_value="extrapolate")
    all_sample_indices = np.arange(total_length)
    smooth_lag_map = lag_interpolator(all_sample_indices)

    # Erstelle eine Interpolationsfunktion für das saubere Signal, um es an nicht-ganzzahligen
    # Positionen abtasten zu können (Time-Warping).
    clean_interpolator = interp1d(np.arange(len(x_clean)), x_clean, kind='cubic', fill_value=0.0, bounds_error=False)

    # Berechne die neuen, verschobenen Sample-Positionen.
    warped_indices = all_sample_indices - smooth_lag_map

    # Erzeuge das ausgerichtete Signal.
    x_clean_aligned = clean_interpolator(warped_indices)

    return x_clean_aligned


### `verify_alignment`
Eine einfache, aber effektive Metrik, um den Erfolg der Ausrichtung zu messen. Sie berechnet die Kreuzkorrelation zwischen dem Zielsignal (Tape) und dem neuen, ausgerichteten Signal. Der Peak der Korrelation sollte idealerweise bei einem Offset von 0 Samples liegen.

In [ ]:
def verify_alignment(x_tape: np.ndarray, x_aligned: np.ndarray) -> int:
    """
    Berechnet den verbleibenden Offset zwischen zwei Signalen via Kreuzkorrelation.
    Ein perfektes Ergebnis ist 0.
    """
    correlation = scipy.signal.correlate(x_tape, x_aligned, mode='same', method='fft')
    offset = np.argmax(correlation) - len(correlation) // 2
    return offset


## 3. Haupt-Workflow: Batch-Verarbeitung

### `find_chunk_pairs`
Diese Funktion durchsucht die Verzeichnisstruktur und erstellt eine Liste von zu verarbeitenden Paaren aus Clean- und Tape-Chunks.

In [ ]:
def find_chunk_pairs(input_dir: Path, tape_dir: Path) -> List[Tuple[Path, Path]]:
    """Findet Paare von (clean_chunk, tape_chunk) basierend auf dem Dateinamen."""
    pairs = []
    print(f"Suche nach Chunks in: {input_dir}")
    for clean_path in sorted(input_dir.rglob('*.wav')):
        relative_path = clean_path.relative_to(input_dir)
        tape_path = tape_dir / relative_path
        if tape_path.exists():
            pairs.append((clean_path, tape_path))
    print(f"Gefunden: {len(pairs)} Paare.")
    return pairs


### `process_chunk`
Diese Funktion kapselt die gesamte Logik für ein einzelnes Chunk-Paar: Laden, Validieren, Lag-Berechnung, Warping, Verifizierung und Speichern. Sie gibt detaillierte Status- und Fehlermeldungen aus.

In [ ]:
def process_chunk(clean_path: Path, tape_path: Path, output_path: Path):
    """Führt den kompletten Ausrichtungsprozess für ein einzelnes Chunk-Paar durch."""
    try:
        # 1. Laden und Sample Rate prüfen
        y_clean, sr_c = librosa.load(clean_path, sr=None, mono=True)
        y_tape, sr_t = librosa.load(tape_path, sr=None, mono=True)

        if sr_c != SR_ORIGINAL or sr_t != SR_ORIGINAL:
            print(f"  SKIPPED: {clean_path.name} (Falsche SR. Erwartet: {SR_ORIGINAL}, gefunden: {sr_c}/{sr_t})")
            return

        # 2. Chunk-Länge validieren
        duration_sec = len(y_clean) / SR_ORIGINAL
        if duration_sec < MIN_CHUNK_DURATION_SEC:
            print(f"  SKIPPED: {clean_path.name} (Zu kurz: {duration_sec:.2f}s < {MIN_CHUNK_DURATION_SEC}s)")
            return

        min_len = min(len(y_clean), len(y_tape))
        y_clean, y_tape = y_clean[:min_len], y_tape[:min_len]

        # 3. Lokalen Versatz berechnen
        times, raw_lags = compute_local_lags_resampled(y_clean, y_tape, SR_ORIGINAL, SR_CALC, WINDOW_SEC, HOP_SEC)

        if len(times) == 0:
            print(f"  SKIPPED: {clean_path.name} (Keine Lags berechenbar, evtl. zu wenig Analysefenster)")
            return

        # 4. Warping anwenden
        y_clean_aligned = apply_warping_full_res(y_clean, raw_lags, times)

        # 5. Ergebnis verifizieren
        final_offset = verify_alignment(y_tape, y_clean_aligned)
        print(f"  SUCCESS: {clean_path.name} -> Verbleibender Versatz: {final_offset} Samples.")

        # 6. Speichern
        output_path.parent.mkdir(parents=True, exist_ok=True)
        sf.write(output_path, y_clean_aligned, SR_ORIGINAL)

    except Exception as e:
        print(f"  ERROR:   {clean_path.name} ({e})")


### Ausführung der Batch-Verarbeitung
Der folgende Code-Block startet den Prozess. Er iteriert durch alle Unterordner im `BASE_INPUT_DIR`, findet die entsprechenden Tape-Ordner, sucht nach Chunk-Paaren und startet die Verarbeitung für jedes Paar. Bereits existierende Output-Dateien werden übersprungen.

In [ ]:
# --- Start der Batch-Verarbeitung ---
if __name__ == '__main__':
    input_sets = [d for d in BASE_INPUT_DIR.iterdir() if d.is_dir()]

    for input_set_dir in input_sets:
        set_name = input_set_dir.name
        print(f"\n--- Verarbeite Set: {set_name} ---")

        tape_set_dir = BASE_TAPE_DIR / f"{set_name}_tape"
        output_set_dir = BASE_OUTPUT_DIR / f"{set_name}_aligned"

        if not tape_set_dir.exists():
            print(f"WARNUNG: Tape-Verzeichnis {tape_set_dir} nicht gefunden. Überspringe Set.")
            continue

        chunk_pairs = find_chunk_pairs(input_set_dir, tape_set_dir)
        processed_count = 0

        for i, (clean_path, tape_path) in enumerate(chunk_pairs):
            relative_path = clean_path.relative_to(input_set_dir)
            output_path = output_set_dir / relative_path

            if output_path.exists():
                continue

            if processed_count > 0 and processed_count % 50 == 0:
                print(f"  ... {processed_count}/{len(chunk_pairs)} Chunks verarbeitet ...")

            process_chunk(clean_path, tape_path, output_path)
            processed_count += 1

        print(f"--- Set {set_name} abgeschlossen. {processed_count} neue Chunks verarbeitet. ---")

    print("\n--- Batch-Verarbeitung vollständig abgeschlossen. ---")
